# 전이 학습(Transfer Learning)

## <span style="font-size:25px">개념 및 필요성</span>

<span style="font-size:22px">1. 개념</span>

- <span style="font-size:18px">전이 학습이란 아주 큰 데이터 셋, 즉 21,841 부류에 대해서 총 1419만 7122장의 이미지로<br> 구성되어 있는 ImageNet 데이터를 사용해서 학습된 모델의 가중치를 가져와서, 우리가 해<br>결하려는 문제에 맞게 보정해서 사용하는 것을 의미한다</span>

- <span style="font-size:18px">이때 큰 데이터 셋을 사용해서 훈련된 모델을 사전 학습 모델(pre-trained model)</span>

&emsp;&emsp;<span style="font-size:15px">※ ImageNet 데이터의 이미지 크기는 평균적으로 469×387이며, 이러한 2만 개 이상의 부류 가운데 
<br>&emsp;&emsp;&emsp;1000 부류만 뽑아서 데이터를 구성하고, 정확도를 높이는 대화가 바로 ImageNet Challange</span>

<span style="font-size:22px">2. 필요성</span>

- <span style="font-size:18px">CNN 모델의 품질을 높이기 위해서, 즉 임의의 데이터에 대해서 정확도는 높이고 overfitting은 줄이기 위해서 기본적으로 많은 양의 데이터를 이용하여 학습하여야 한다</span>

- <span style="font-size:18px">그러나 많은 학습 데이터를 확보하려면 시간과 비용이 많이 소요되기에, 이 부분을 해결하고자 등장한 것이 전이 학습이다

## <span style="font-size:25px">사전 학습 모델(pre-trained model) 구조</span>

<table>
<tr>
<td style="font-size:17px">
▶ 사전 학습된 특징 추출기<br>

- 특징 추출기(feature extractor)는 convolution <br>layer과 pooling layer의 조합으로 구성되어 있<br>으며 ImageNet 데이터에 대해 이미 학습되어 있음
</td>
<td style="font-size:17px">
<span style="font-size:20px">Conv ⇒ Conv ⇒ Pooling ⇒</span>

※ 특징 추출기의 출력 데이터를 bottleneck 또는 <br>feature vector 등으로 지정함
</td>
</tr>

<tr>
<td style="font-size:17px">
▶ 사전 학습된 분류기<br>

- 분류기(classifier)는 fully-connected layer으로 <br>구성되며 추출된 특징을 입력 받아 최종 주어<br>지는 이미지에 대한 클래스(정답)을 카테고리 <br>형태로 분류하는 역할 수행
</td>
<td style="font-size:17px">
<span style="font-size:20px">Dense ⇒ Dense ⇒ Dense(softmax) ⇒</span>

※ overfitting을 줄이기 위해 output layer 이전의 <br>Dense layer 사이에는 Dropout, BatchNormalization <br>layer 등을 add 할 수 있음
</td>
</tr>
</table>

## <span style="font-size:25px">사전 학습 모델(pre-trained model) 종류</span>

<span style="font-size:18px">TensorFlow가 제공하는 pre-trained model 이름과 크기, ImageNet 데이터에 대한<br> 1순위와 5순위 정확도, 파라미터 개수 및 층의 깊이 등을 나타낸다</span>

- <span style="font-size:17px">사용할 데이터 셋에 따라 모델의 성능이 다르기 때문에 모델 간의 비교가 중요</span>

<img src="picture/TF_img_25_1.png" alt="TF_img_25_1" width=850>

## <span style="font-size:25px">파인 튜닝(fine-tuning)</span>

<span style="font-size:18px">pre-trained model의 가중치를 미세하게 조정하는 기법
<br>new data의 일부 혹은 전체를 미리 분석한 후, 이를 베이스로 pre-trained model 가중치 일부 또는 전체 가중치를 재학습 하는 것이다</span>

- <span style="font-size:17px">fine-tuning 시 많은 연산량이 필요하기에 GPU를 사용하는 것이 좋다</span>

## <span style="font-size:25px">Transfer Learning 사용법</span>

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import VGG16, ResNet50, MobileNet, InceptionV3

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(240, 240, 3))
#                 1. 사전학습에 사용된 데이터 셋
#                                     2. False 사전 학습 모델의 특징추출기만 가져옴
#                                        True 특징추출기와 분류기 모두 가져옴
#                                                         3. 새롭게 학습할 이미지 텐서 크기
base_model.summary()

58889256/58889256 [==============================] - 6s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 240, 240, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 240, 240, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 240, 240, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 120, 120, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 120, 120, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 120, 120, 128)     147584    
                                                              

In [4]:
model = Sequential()
# 이전 학습된 내용 추출
model.add(base_model)
model.add(Flatten())    # Flatten() or GlobalAveragePooling2D()
                        # 변환
# 새로운 데이터 학습 및 이전 학습과 새로운 학습을 연결
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dense_4 (Dense)             (None, 64)                1605696   
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 4)                 260       
                                                                 
Total params: 16,320,644
Trainable params: 16,320,644
Non-trainable params: 0
_________________________________________________________________
